In [80]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import os
os.environ["OMP_NUM_THREADS"] = '1'
import warnings
warnings.filterwarnings('ignore')

In [81]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408


In [82]:
# Generate summary statistics
df_market_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [83]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

---

### Prepare the Data

In [84]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
scaler = StandardScaler()


In [85]:
# Create a DataFrame with the scaled data
scaled_df = scaler.fit_transform(df_market_data)

# Copy the crypto names from the original DataFrame
scaled_df=pd.DataFrame(scaled_df,columns=df_market_data.columns)

# Set the coin_id column as index
scaled_df['coin_id']= df_market_data.index

# Display the scaled DataFrame
scaled_df.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y,coin_id
0,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637,bitcoin
1,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352,ethereum
2,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061,tether
3,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546,ripple
4,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317,bitcoin-cash


---

### Find the Best Value for k Using the Original Scaled DataFrame.

In [86]:
# Create a new DataFrame that holds the numeric columns from the original DataFrame
df_market_data_scaled = scaled_df.drop(columns=["coin_id"])

In [87]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1, 11))

In [88]:
# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list
for i in k:
    model = KMeans(n_clusters=i, random_state=1)
    model.fit(df_market_data_scaled)
    inertia.append(model.inertia_)


In [89]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {"k": k, "inertia": inertia}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow = pd.DataFrame(elbow_data)

In [90]:
# Plot a line chart with all the inertia values computed with
# the different values of k to visually identify the optimal value for k.
df_elbow.hvplot.line(
    x="k",
    y="inertia",
    title="Elbow Curve",
    xticks=k,
    xlabel="Number of Clusters (k)",
    ylabel="Inertia"
)

:Curve   [k]   (inertia)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** The best value for'k' is 4

---

### Cluster Cryptocurrencies with K-means Using the Original Scaled DataFrame

In [91]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4, random_state=1)

In [92]:
# Fit the K-Means model using the scaled DataFrame
model.fit(df_market_data_scaled)

KMeans(n_clusters=4, random_state=1)

In [93]:
# Predict the clusters to group the cryptocurrencies using the scaled DataFrame
predictions = model.predict(df_market_data_scaled)

# Print the resulting array of cluster values.
print(predictions)

[2 2 0 0 2 2 2 2 2 0 0 0 0 2 0 2 0 0 2 0 0 2 0 0 0 0 0 0 2 0 0 0 3 2 0 0 1
 0 0 0 0]


In [94]:
# Create a copy of the scaled DataFrame
df_predictions = df_market_data_scaled.copy()

In [95]:
# Add a new column to the copy of the scaled DataFrame with the predicted clusters
df_predictions["Cluster"] = predictions

# Display the copy of the scaled DataFrame
df_predictions.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y,Cluster
0,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637,2
1,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352,2
2,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061,0
3,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546,0
4,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317,2


In [96]:
# Create a scatter plot using hvPlot by setting
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`.
# Color the graph points with the labels found using K-Means and
# add the crypto name in the `hover_cols` parameter to identify
# the cryptocurrency represented by each data point.
df_predictions.hvplot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    by="Cluster",
    hover_cols=["coin_id"],
    title="Cryptocurrency Analysis"
)

:NdOverlay   [Cluster]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d)

---

### Optimize Clusters with Principal Component Analysis.

In [97]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=3)

In [98]:
# Use the PCA model with `fit_transform` to reduce the original scaled DataFrame
# down to three principal components.
crypto_data = pca.fit_transform(df_market_data_scaled)

# View the scaled PCA data
print(crypto_data[:5])

[[-0.60066733  0.84276006  0.46159457]
 [-0.45826071  0.45846566  0.95287678]
 [-0.43306981 -0.16812638 -0.64175193]
 [-0.47183495 -0.22266008 -0.47905316]
 [-1.15779997  2.04120919  1.85971527]]


In [99]:
# Retrieve the explained variance to determine how much information
# can be attributed to each principal component.
pca.explained_variance_ratio_

array([0.3719856 , 0.34700813, 0.17603793])

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** The total explained variance is 90% of the three principal components.

In [100]:
# Create a new DataFrame with the PCA data.
# Copy the crypto names from the original scaled DataFrame
df_crypto_pca = pd.DataFrame(data=crypto_data, columns=["PC1", "PC2", "PC3"])

# Set the coin_id column as index
df_crypto_pca.index = df_market_data.index

# Display the scaled PCA DataFrame
df_crypto_pca.head()

,PC1,PC2,PC3
coin_id,,,
bitcoin,-0.600667,0.842760,0.461595
ethereum,-0.458261,0.458466,0.952877
tether,-0.433070,-0.168126,-0.641752
ripple,-0.471835,-0.222660,-0.479053
bitcoin-cash,-1.157800,2.041209,1.859715


---

### Find the Best Value for k Using the Scaled PCA DataFrame

In [101]:
#Create a DataFrame with the principle components data
df_market_data_pca = pd.DataFrame(df_crypto_pca)

In [102]:
# Create a list with the number of k-values from 1 to 11
k_pca = range(1, 11)

In [103]:
# Create an empty list to store the inertia values
inertia_pca = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_pca`
# 3. Append the model.inertia_ to the inertia list
for i in k_pca:
    model = KMeans(n_clusters=i, random_state=1)
    model.fit(df_market_data_pca)
    inertia_pca.append(model.inertia_)

In [104]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data_2 = {"k_pca": k_pca, "inertia_pca": inertia_pca}
# Create a DataFrame with the data to plot the Elbow curve
df_elbow_2 = pd.DataFrame(elbow_data_2)

# Review the DataFrame
df_elbow_2.head()

,k_pca,inertia_pca
0,1,256.874086
1,2,182.339530
2,3,135.442408
3,4,49.665497
4,5,38.672582


In [105]:
# Plot a line chart with all the inertia values computed with
# the different values of k to visually identify the optimal value for k.
df_elbow_2.hvplot.line(
    x="k_pca",
    y="inertia_pca",
    title="Elbow Curve 2",
    xticks=list(k),  # Convert k (range object) to a list to avoid errors
    xlabel="Number of Clusters (k)",
    ylabel="Inertia"
)


:Curve   [k_pca]   (inertia_pca)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** The best value for 'k' when using the PCA data is 4.


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** No it doesn't differ from the best k value found using the original data.

### Cluster Cryptocurrencies with K-means Using the Scaled PCA DataFrame

In [106]:
# Initialize the K-Means model using the best value for k
model_pca = KMeans(n_clusters=4,random_state=1)

In [107]:
# Fit the K-Means model using the PCA data
model_pca.fit(df_market_data_pca)

KMeans(n_clusters=4, random_state=1)

In [108]:
# Predict the clusters to group the cryptocurrencies using the scaled PCA DataFrame
pca_predictions = model_pca.predict(df_market_data_pca)

# Print the resulting array of cluster values.
pca_predictions

array([2, 2, 0, 0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 2, 0, 2, 0, 0, 2, 0, 0, 2,
       0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 3, 2, 0, 0, 1, 0, 0, 0, 0])

In [109]:
# Create a copy of the scaled PCA DataFrame
scaled_pca_df = df_market_data_pca.copy()

# Add a new column to the copy of the PCA DataFrame with the predicted clusters
scaled_pca_df['Clusters']=pca_predictions

# Display the copy of the scaled PCA DataFrame
scaled_pca_df.head()

,PC1,PC2,PC3,Clusters
coin_id,,,,
bitcoin,-0.600667,0.842760,0.461595,2
ethereum,-0.458261,0.458466,0.952877,2
tether,-0.433070,-0.168126,-0.641752,0
ripple,-0.471835,-0.222660,-0.479053,0
bitcoin-cash,-1.157800,2.041209,1.859715,2


In [110]:
# Create a scatter plot using hvPlot by setting
# `x="PC1"` and `y="PC2"`.
# Color the graph points with the labels found using K-Means and
# add the crypto name in the `hover_cols` parameter to identify
# the cryptocurrency represented by each data point.
scaled_pca_df.hvplot.scatter(
    x="PC1",
    y="PC2",
    by="Clusters",
    hover_cols=["coin_id"],
    title="Cryptocurrency Analysis with PCA")

:NdOverlay   [Clusters]
   :Scatter   [PC1]   (PC2,coin_id)

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [114]:
elbow_curve_original = df_elbow.hvplot.line(
    x="k",
    y="inertia",
    title="Elbow Curve - Original Data",
    xlabel="Number of Clusters (k)",
    ylabel="Inertia",
    line_width=2,
    color='blue'
)

elbow_curve_pca = df_elbow_2.hvplot.line(
    x="k_pca",
    y="inertia_pca",
    title="Elbow Curve - PCA Data",
    xlabel="Number of Clusters (k)",
    ylabel="Inertia",
    line_width=2,
    color='green'
)

# Combine the plots side by side using '+' and `cols` layout for better visualization
(elbow_curve_original + elbow_curve_pca).cols(2)

:Layout
   .Curve.I  :Curve   [k]   (inertia)
   .Curve.II :Curve   [k_pca]   (inertia_pca)

In [ ]:
#elbow_curve_original * elbow_curve_pca

:Overlay
   .Curve.I  :Curve   [k]   (inertia)
   .Curve.II :Curve   [k_pca]   (inertia_pca)

In [117]:
# Composite plot to contrast the clusters

clusters_original = df_predictions.hvplot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    by="Cluster",
    hover_cols=["coin_id"],
    title="Cryptocurrency Analysis"
)

clusters_pca = scaled_pca_df.hvplot.scatter(
    x="PC1",
    y="PC2",
    by="Clusters",
    hover_cols=["coin_id"],
    title="Cryptocurrency Analysis with PCA")

(clusters_original + clusters_pca).cols(2)

:Layout
   .NdOverlay.I  :NdOverlay   [Cluster]
      :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d)
   .NdOverlay.II :NdOverlay   [Clusters]
      :Scatter   [PC1]   (PC2,coin_id)

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** By reducing the number of features with PCA, the clusters became more distinct and visually clearer. The elbow curve displayed a more pronounced drop, simplifying the process of selecting the optimal number of clusters. Eliminating irrelevant information sped up the clustering process and improved focus. Overall, this approach made the results more straightforward and easier to interpret.